In [33]:
import sys
sys.path.append('../')
import numpy as np
from src.utils_py.gro.Structure import Structure
from src.utils_py.gro.Atom import Atom_label
from src.utils_py.io.gro import write_gro, read_gro

---

move O through PBC

In [17]:
# structure = read_gro('../structures/calcite/calcite_104_unitcell.gro')
structure = read_gro('../substrates/calcite/CaCO3_104_rotated_cell_fixed.gro')

In [18]:
z_dir = np.array([structure.box[-2], structure.box[-1], structure.box[2]])
z_dir_norm = z_dir / np.linalg.norm(z_dir)
print(z_dir)
print('norm:', np.linalg.norm(z_dir))

[ 0.01586 -0.       1.21429]
norm: 1.2143935703469448


In [19]:
structure.atoms_xyz += z_dir_norm * 0.1

In [21]:
with open('../substrates/calcite/calcite_104_unitcell_test.gro', 'w') as f:
    f.write(write_gro(structure.apply_pbc()))

---

center unitcell

In [22]:
def proj(a, b):
    return np.dot(a, b) / np.linalg.norm(a)**2 * a

In [24]:
structure = read_gro('../substrates/calcite/calcite_104_unitcell_test.gro')

In [25]:
z_dir = np.array([structure.box[-2], structure.box[-1], structure.box[2]])
z_dir_norm = z_dir / np.linalg.norm(z_dir)
print(z_dir)
print('norm:', np.linalg.norm(z_dir))

[ 0.01586 -0.       1.21429]
norm: 1.2143935703469448


In [26]:
alpha = np.arccos(np.dot(z_dir_norm, np.array([0, 0, 1])))
print(np.rad2deg(alpha))

0.7483050960465677


In [27]:
max_id = 48
min_id = 34

In [28]:
z_max = structure.atoms_xyz[max_id-1, 2]
z_min = structure.atoms_xyz[min_id-1, 2]
print(z_max, z_min)

1.089 0.022


In [29]:
structure.atoms_xyz += z_dir_norm * ((z_dir[2] - z_max) - z_min) / 2 / np.cos(alpha)

In [31]:
with open('../substrates/calcite/calcite_104_unitcell_test.gro', 'w') as f:
    f.write(write_gro(structure.apply_pbc()))

---

renaming atoms

In [35]:
structure = read_gro('../substrates/calcite/calcite_104_unitcell.gro')

In [41]:
counter_dict = {"CA": 0, "OCA": 0, "CCA": 0}
for i, label in np.ndenumerate(structure.atoms):
    name = "".join([i for i in label.name if not i.isdigit()])

    counter_dict[name] += 1
    if label.name == "CCA":
        atom_name = "C" + str(counter_dict[name])
    elif label.name == "OCA":
        atom_name = "O" + str(counter_dict[name])
    elif label.name == "CA":
        atom_name = "Ca" + str(counter_dict[name])

    structure.atoms[i].id = i[0]+1
    structure.atoms[i[0]].name = atom_name

In [42]:
with open('../substrates/calcite/calcite_104_unitcell_test.gro', 'w') as f:
    f.write(write_gro(structure.apply_pbc()))